In [1]:
import sys
import math
import import_ipynb
import os
from Generator_DotProduct_N_SI_HR_v4_6 import DotProduct_SI

importing Jupyter notebook from Generator_DotProduct_N_SI_HR_v4_6.ipynb
importing Jupyter notebook from DotProduct_SoE_V4_5.ipynb
importing Jupyter notebook from DotProduct_Systolic_V1_11.ipynb
importing Jupyter notebook from DotProduct_noHR_M_V3_7.ipynb
importing Jupyter notebook from RegisteredNto1Adder_V1_1.ipynb
importing Jupyter notebook from DotProduct_noHR_C_V3_7.ipynb
importing Jupyter notebook from AdderTree_ASm_V1_20.ipynb
importing Jupyter notebook from SItoSoE_V1_10.ipynb


In [4]:
def DotProduct_SI_AXI_Stream(Parameters,Print_To_File,Files_Location,Is_Top,modules):
    #N>=2
    N = input('Enter N: ')
    N = int(N)
    if N<2:
        N = 2
    #HRR>=2
    HRR = input('Enter HRR Upper Limit (>=2, <={}, empty for 2, not all HRR values are acceptable): '.format(N))
    if not HRR:
        HRR = 2
    else:
        HRR = int(HRR)
        if HRR<2:
            HRR = 2
        if HRR>N: 
            HRR = N

    NMult = math.ceil(N/HRR)
    #print("NMult: {}".format(NMult))

    while math.ceil(N/NMult) != HRR:
        HRR -= 1
    print("HRR calculated value: {}".format(HRR))
    #MAMCS>=1
    MAMCS = input('Enter Multiply-Add Max Component Size (empty for {}): '.format(NMult))
    if not MAMCS:
        MAMCS = NMult
    else:
        MAMCS = int(MAMCS)
        if MAMCS>NMult:
            MAMCS = NMult
        elif MAMCS<1:
            MAMCS = 1
    #AdderSize>=2
    if MAMCS<NMult:
        AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')
    ENABLE_INPUT_LATCH = input('Enable Input Latch? (empty or 0 to disable, 1 to enable): ')
    #IN_WIDTH>=1
    IN_WIDTH = input('Enter IN_WIDTH Default Value (empty for 10): ')
    #INPUT_REG_DEPTH>=0
    INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH Default Value (empty for 1): ')
    #MULT_PIPE_DEPTH>=0
    MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH Default Value (empty for 1): ')
    NumOfComponents = math.ceil(NMult/MAMCS)
    if MAMCS==NMult:
        AdderSize = 0
    else:
        if not AdderSize:
            AdderSize = 2
        else:
            AdderSize = int(AdderSize)
            if AdderSize<2:
                AdderSize = 2
            if AdderSize>NumOfComponents:
                AdderSize = NumOfComponents

    if not IN_WIDTH:
        IN_WIDTH=10
    else:
        IN_WIDTH = int(IN_WIDTH)
        if(IN_WIDTH<1):
            IN_WIDTH=1

    if not INPUT_REG_DEPTH:
        INPUT_REG_DEPTH=1
    else:
        INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)
        if(INPUT_REG_DEPTH<0):
            INPUT_REG_DEPTH=0

    if not MULT_PIPE_DEPTH:
        MULT_PIPE_DEPTH=1
    else:
        MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)
        if(MULT_PIPE_DEPTH<0):
            MULT_PIPE_DEPTH=0

    if not ENABLE_INPUT_LATCH:
        ENABLE_INPUT_LATCH=0
    else:
        ENABLE_INPUT_LATCH = int(ENABLE_INPUT_LATCH)

    parameters={ 
        "N":N,
        "HRR":HRR,
        "MAMCS":MAMCS,
        "ADDER_SIZE":AdderSize,
        "IN_WIDTH":IN_WIDTH,
        "INPUT_REG_DEPTH":INPUT_REG_DEPTH,
        "MULT_PIPE_DEPTH":MULT_PIPE_DEPTH,
       # "ENABLE_INPUT_REGISTERS":ENABLE_INPUT_REGISTERS,
        "ENABLE_INPUT_LATCH":ENABLE_INPUT_LATCH,
        "IN_WIDTH":IN_WIDTH
    }
    lgN=math.ceil(math.log2(N))
    IN_DATA_LENGHT=2*N*IN_WIDTH    
    OUT_WIDTH=2*IN_WIDTH+lgN
    #OUT_DATA_LENGHT_HR=NAdd*OUT_WIDTH
    OUT_DATA_LENGHT=OUT_WIDTH
    #OUT_DATA_LENGHT_NE=NE*OUT_WIDTH

    Print_To_File=0
    Files_Location='DotProduct_GeneratedVerilogCodes/'
    Is_Top=1
    modules={}
    GeneratedFileName=DotProduct_SI(parameters,Print_To_File,Files_Location,Is_Top,modules)
    Files_Location+=GeneratedFileName+"/"
    wrapperModuleName=GeneratedFileName+"_AXIStream"
    of=open(Files_Location+wrapperModuleName+".v", 'w+')

    # in this cell we will use the generated file and instanciate it in our new AXI wrapper module.
    print("`timescale 1ns / 1ps\n",file=of)
    print("module "+wrapperModuleName,file=of)
    print("#(",file=of)
    print("parameter IN_DATA_LENGHT= {}, ".format(IN_DATA_LENGHT),file=of)
    print("parameter OUT_DATA_LENGHT= {} ".format(OUT_DATA_LENGHT),file=of)
    print(")( \n",file=of)
    print("input aclk,".format(),file=of)
    print("input aresetn,".format(),file=of)
    print("input enable,".format(),file=of)
    print("input [IN_DATA_LENGHT-1:0]s_axi_data,".format(),file=of)
    print("input s_axi_valid,".format(),file=of)
    print("input m_axi_ready,".format(),file=of)
    print("output reg[OUT_DATA_LENGHT-1:0]m_axi_data,".format(),file=of)
    print("output reg m_axi_valid,".format(),file=of)
    print("output reg s_axi_ready".format(),file=of)
    print(");\n".format(),file=of)
    print("reg inready;".format(),file=of)
    print("reg [IN_DATA_LENGHT-1:0]indata;".format(),file=of)
    print("wire outready;".format(),file=of)
    print("wire earlyoutready;".format(),file=of)
    print("wire [OUT_DATA_LENGHT-1:0]outdata;".format(),file=of)

    print("////////////////instancing wrapping module///////////////////".format(),file=of)
    print("{} {}(".format(GeneratedFileName,GeneratedFileName+"_inst"),file=of)
    print(".clk(aclk),".format(),file=of)
    print(".reset(!aresetn),".format(),file=of)
    print(".enable(enable),".format(),file=of)
    print(".inReady(inready),".format(),file=of)
    for i in range (N):
        print(".A{}(indata[{}:{}]),".format( i, IN_DATA_LENGHT-(i*IN_WIDTH)-1  ,  IN_DATA_LENGHT-(i+1)*IN_WIDTH), file=of )
    for i in range (N):    
        print(".B{}(indata[{}:{}]),".format( i, int((IN_DATA_LENGHT)/2-(i*IN_WIDTH)-1) , int(IN_DATA_LENGHT/2-(i+1)*IN_WIDTH)), file=of )
    print(".outReady(outready),".format(), file=of)
    print(".DP(outdata),".format(), file=of)
    print(".earlyOutReady(earlyoutready)".format(),file=of)
    print(");".format(),file=of)

    #in this cell we will add Axi stream compatability codes to wrapp the generated module in the previeous cell
    print("/////////////////Main body/////////////".format(),file=of)
    print("always @(posedge aclk)begin".format(),file=of)
    print(" if(aresetn==0)begin".format(),file=of)
    print("  m_axi_data<=0;".format(),file=of)
    print("  m_axi_valid<=0;".format(),file=of)
    print(" end".format(),file=of)
    print(" else begin".format(),file=of)
    print("  if(m_axi_ready==1 && m_axi_valid==1)begin".format(),file=of)
    print("   m_axi_valid<=0;".format(),file=of)
    print("  end".format(),file=of)
    print("  else if(outready==1)begin".format(),file=of)
    print("   m_axi_valid<=1;".format(),file=of)
    print("   m_axi_data<=outdata;".format(),file=of)
    print("  end".format(),file=of)
    print(" end".format(),file=of)
    print("end".format(),file=of)
    print("always @(posedge aclk)begin".format(),file=of)
    print(" if(aresetn==0)begin".format(),file=of)
    print("  s_axi_ready<=1;".format(),file=of)
    print("  inready<=0;".format(),file=of)
    print("  indata<=0;".format(),file=of)
    print(" end".format(),file=of)
    print(" else begin".format(),file=of)
    print("  inready<=0;".format(),file=of)
    print("  if(s_axi_valid==1 && s_axi_ready==1)begin".format(),file=of)
    print("   s_axi_ready<=0;".format(),file=of)
    print("   inready<=1;".format(),file=of)
    print("   indata<= s_axi_data;".format(),file=of)
    print("  end".format(),file=of)
    print("  else if(m_axi_valid==1 && m_axi_ready==1)begin".format(),file=of)
    print("   s_axi_ready<=1;".format(),file=of)
    print("  end".format(),file=of)
    print(" end".format(),file=of)
    print("end".format(),file=of)
    of.close()
    return wrapperModuleName

In [5]:
DotProduct_N_SI_HR_v4_6_AXI_Stream()

Enter N: 10
Enter HRR Upper Limit (>=2, <=10, empty for 2, not all HRR values are acceptable): 
HRR calculated value: 2
Enter Multiply-Add Max Component Size (empty for 5): 
Enable Input Latch? (empty or 0 to disable, 1 to enable): 
Enter IN_WIDTH Default Value (empty for 10): 
Enter INPUT_REG_DEPTH Default Value (empty for 1): 
Enter MULT_PIPE_DEPTH Default Value (empty for 1): 
HRR calculated value: 2
`timescale 1ns / 1ps

module DotProduct_10_SI_HRx2_C5_A0_NIL
#(parameter
IN_WIDTH = 10,
INPUT_REG_DEPTH = 1,
MULT_PIPE_DEPTH = 1
)(
input clk, reset, enable,
output newInSeriesStart,
output inSeries,
input inReady,
input signed [IN_WIDTH-1:0] A0, A1, A2, A3, A4, A5, A6, A7, A8, A9, 
input signed [IN_WIDTH-1:0] B0, B1, B2, B3, B4, B5, B6, B7, B8, B9, 
output outReady,
output signed [(2*IN_WIDTH)+3:0] DP,
output earlyOutReady
);

wire DPInReady;
wire signed [IN_WIDTH-1:0] AS0, AS1, AS2, AS3, AS4;
SItoSoE_10_HRx2_NIL_NOR #( .IN_WIDTH(IN_WIDTH) )
SItSoEA (
clk, reset, enable,
newInSeriesSta

'DotProduct_10_SI_HRx2_C5_A0_NIL_AXIStream'